# DeliverableValidator Tutorial

The **DeliverableValidator** is the simplest agent pattern in agent-workshop - a single-message agent that validates documents for completeness and quality.

## Overview

- **Type**: Simple Agent (single input/output)
- **Use Case**: Document validation, report review, content assessment
- **Features**: Presets for different domains, multi-layer configuration

## Setup

In [ ]:
from agent_workshop.agents.validators import DeliverableValidator
from agent_workshop.agents.validators.presets import get_preset, list_presets, get_preset_info
from agent_workshop import Config

config = Config()
validator = DeliverableValidator(config)

print(f"Provider: {validator.provider_name}")
print(f"Model: {validator.model_name}")

## Available Presets

DeliverableValidator comes with pre-configured presets for different document types:

In [ ]:
# List all available presets
print("Available Presets:")
print("=" * 50)
for preset_name in list_presets():
    info = get_preset_info(preset_name)
    print(f"\n{preset_name}:")
    print(f"  Criteria count: {info['criteria_count']}")
    print(f"  Output format: {info['output_format']}")
    print(f"  First criterion: {info['first_criterion'][:50]}...")

In [ ]:
# View a specific preset's details
financial_preset = get_preset("financial_report")

print("Financial Report Preset:")
print("=" * 50)
print("\nValidation Criteria:")
for i, criterion in enumerate(financial_preset['validation_criteria'], 1):
    print(f"  {i}. {criterion}")

## Configuration Layers

DeliverableValidator supports multiple configuration sources (highest to lowest priority):

1. **Constructor parameters** - Direct code configuration
2. **YAML config file** - prompts.yaml in project root
3. **Environment variables** - VALIDATOR_SYSTEM_PROMPT, VALIDATOR_CRITERIA
4. **Preset** - Pre-built configurations
5. **Defaults** - Built-in fallback values

## Input/Output Format

**Input**: Document content as a string

**Output**:
```python
{
    "validation": str,  # Detailed validation feedback
    "timestamp": str    # ISO format timestamp
}
```

The validation feedback includes:
- Overall assessment (approved/needs_revision)
- Specific strengths
- Improvement recommendations
- Priority actions

## Example: Validate a Financial Report

In [ ]:
# Sample financial report with issues
sample_report = '''
Q3 2024 Financial Summary

Revenue: $2.4M (up 15% YoY)
Operating Expenses: $1.8M
Net Income: $0.6M

Key Highlights:
- Strong customer acquisition in enterprise segment
- Reduced churn rate to 3.2%
- Launched new product line generating $200K

Notes:
- Numbers are preliminary and subject to audit
- Cash position remains healthy
'''

print("Report to validate:")
print(sample_report)

In [ ]:
# Create validator with financial preset
preset = get_preset("financial_report")
financial_validator = DeliverableValidator(config, **preset)

print("Financial Validator Configuration:")
print(f"  Criteria: {len(financial_validator.validation_criteria)} items")
print(f"  Output format: {financial_validator.output_format}")

In [ ]:
# Run the validation (uncomment to execute)
# result = await financial_validator.run(sample_report)
#
# print("Validation Result:")
# print("=" * 50)
# print(result.get('validation'))
# print(f"\nTimestamp: {result.get('timestamp')}")

## Example: Research Paper Validation

In [ ]:
# Create validator with research paper preset
research_preset = get_preset("research_paper")
research_validator = DeliverableValidator(config, **research_preset)

sample_abstract = '''
This study examines the correlation between remote work adoption and employee productivity.

Methodology: We surveyed 500 employees across 20 companies over 6 months.

Findings: Remote workers showed 12% higher task completion rates but
23% lower collaboration scores.

Implications: Organizations should balance remote flexibility with
structured collaboration time.
'''

print("Research abstract to validate:")
print(sample_abstract)
print("\nValidation Criteria:")
for c in research_validator.validation_criteria[:3]:
    print(f"  - {c[:60]}...")

## Custom Configuration

You can fully customize the validator with your own criteria:

In [ ]:
# Custom validator for API documentation
api_validator = DeliverableValidator(
    config=config,
    system_prompt="""You are an API documentation reviewer.
    Focus on clarity, completeness, and developer experience.
    Ensure code examples are runnable and well-formatted.""",
    validation_criteria=[
        "All endpoints have clear descriptions",
        "Request/response examples are provided",
        "Authentication requirements are documented",
        "Error codes and messages are listed",
        "Rate limits are specified",
    ],
    output_format="json"
)

print("Custom API Documentation Validator:")
print(f"  Output format: {api_validator.output_format}")
print("  Criteria:")
for i, c in enumerate(api_validator.validation_criteria, 1):
    print(f"    {i}. {c}")

## YAML Configuration

You can also configure the validator using a YAML file:

```yaml
# prompts.yaml
deliverable_validator:
  system_prompt: |
    You are a document validator focused on clarity and completeness.
    Provide actionable feedback.

  validation_criteria:
    - Clear executive summary
    - Logical flow of information
    - Proper citations and references

  output_format: detailed
```

Then load it:
```python
validator = DeliverableValidator(config, config_file="prompts.yaml")
```

## Output Formats

The validator supports three output formats:

| Format | Description | Best For |
|--------|-------------|----------|
| **detailed** | Full narrative feedback | Human review |
| **json** | Structured JSON response | Automation, parsing |
| **summary** | Brief overview only | Quick checks |

In [ ]:
# Create validators with different output formats
detailed_validator = DeliverableValidator(config, output_format="detailed")
json_validator = DeliverableValidator(config, output_format="json")
summary_validator = DeliverableValidator(config, output_format="summary")

print("Output Format Configurations:")
print(f"  Detailed: {detailed_validator.output_format}")
print(f"  JSON: {json_validator.output_format}")
print(f"  Summary: {summary_validator.output_format}")

## Preset Reference

| Preset | Use Case | Key Focus |
|--------|----------|----------|
| **financial_report** | Financial statements, investor reports | GAAP/IFRS compliance, accuracy |
| **research_paper** | Academic papers, studies | Methodology, citations, reproducibility |
| **technical_spec** | API docs, system design | Completeness, implementability |
| **marketing_content** | Marketing materials | Brand alignment, CTA effectiveness |
| **legal_document** | Contracts, policies | Clarity, enforceability |
| **general** | Any document | Basic quality checks |

## Next Steps

- **[02_validation_pipeline.ipynb](./02_validation_pipeline.ipynb)** - Multi-step validation workflow
- **[03_code_reviewer.ipynb](./03_code_reviewer.ipynb)** - Code review agent
- **[00_getting_started.ipynb](./00_getting_started.ipynb)** - Framework overview